In [ ]:
import numpy as np
import sep

from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Ellipse

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

# Read the FITS file using Astropy
with fits.open("../data/image.fits") as hdul:
    data = hdul[0].data  # Access the primary HDU's data

# Display the image
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()
plt.savefig('figureone.png')

# Estimate the background
bkg = sep.Background(data)

# Advanced background estimation with parameters (mask example is placeholder, ensure 'mask' is defined if used)
# bkg = sep.Background(data, mask=mask, bw=64, bh=64, fw=3, fh=3)

print(bkg.globalback)
print(bkg.globalrms)

# Background image
bkg_image = bkg.back()
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig('figuretwo.png')

# Background RMS image
bkg_rms = bkg.rms()
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig('figthree.png')

# Subtract the background
data_sub = data - bkg

# Object detection
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

print(f"Number of objects detected: {len(objects)}")

# Plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# Plot an ellipse for each detected object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('figurefour.png')

# Access object properties
print(objects.dtype.names)

# Calculate flux and errors
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

# Print flux for up to 10 objects
for i in range(min(10, len(flux))):
    print(f"Object {i}: flux = {flux[i]:.2f} +/- {fluxerr[i]:.2f}")

ModuleNotFoundError: No module named 'astropy'